# Kaggle Playground - Season 4 Episode 
## Binary Classification of Insurance Cross Selling

Competion link - https://www.kaggle.com/competitions/playground-series-s4e8

### Steps
- Import the necessary libraries, packages and modules
- Read the datsets as data framers

### Understand the problem

- class is the target variable
- It determines the class of a mushroom depending on the given variables

In [2]:
# Import the necessary libraries, packages and modules

import warnings
warnings.filterwarnings('ignore')

# import dtale    # Use of a web progrm to analysis the data deeply
import lightgbm as lgb
import logging
import matplotlib.pyplot as plt
import numpy as np
import optuna
import os
import pandas as pd
import pickle
import seaborn as sns
# import statsmodels.api as sm
import tensorflow as tf
import time
import xgboost as xgb
# import zipfile

from imblearn.over_sampling import RandomOverSampler
from optuna.samplers import TPESampler
#from pandas_profiling import ProfileReport
from scipy.stats import randint, uniform
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, HistGradientBoostingClassifier)
from sklearn.feature_selection import chi2, RFE, SelectKBest, SelectFromModel  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# from skopt import BayesSearchCV
from xgboost import XGBClassifier

sns.set()
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
start_time = time.time()

In [3]:
# Test to see if TensorFlow can utilize the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
train_df = pd.read_csv('train.csv', index_col = 'id')
test_df = pd.read_csv('test.csv', index_col = 'id')

train_df.head()

logger.info(f"Train data load completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Train data load completed. Time elapsed: 7.06 seconds


In [5]:
test_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a


In [6]:
# # Checking for data imbalance in training dataframe

# temp = train_df['class'].value_counts()    # Counts the number of ones and zeroes in the column
# temp_df = pd.DataFrame({'class' : temp.index, 'values' : temp.values})    # Creating a data frame with value and their counts
# plt.figure(figsize = (4, 3))
# sns.barplot(x = 'class', y = 'values', data = temp_df)
# plt.show()

In [7]:
# plt.pie(temp_df['values'], labels = [0, 1])
# plt.show()

### Identify the target variable and features

- class is the target variable
- It determines the class of a mushroom depending on the given variables

### Remove duplicate rows

- Checked the sum of duplicated rows in train and test datasets
- No dupllicated rows in train dataframe

In [8]:
# train_df.duplicated().sum()

In [9]:
# test_df.duplicated().sum()

### Handling missing values

- Checked the missing values in column
- There are considerable amount of missing values in many columns

In [10]:
train_df.shape

(3116945, 21)

In [11]:
test_df.shape

(2077964, 20)

In [12]:
# train_df.info()

In [13]:
# test_df.info()

In [14]:
# Since we have only one data set, spliting it into train and test (validation)

raw_train_df, validation_df = train_test_split(train_df, train_size = 0.75, random_state = 1, stratify = train_df['class'])
logger.info(f"Train test split completed. Time elapsed: {time.time() - start_time:.2f} seconds")
raw_train_df.head(2)

INFO:__main__:Train test split completed. Time elapsed: 14.64 seconds


,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
748879,p,2.45,x,g,n,f,a,NaN,p,7.12,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
589395,p,2.71,b,NaN,k,f,a,NaN,k,7.78,...,NaN,NaN,g,NaN,w,f,f,k,g,u


In [15]:
raw_train_df.isnull().sum()

class                         0
cap-diameter                  3
cap-shape                    30
cap-surface              503295
cap-color                    10
does-bruise-or-bleed          6
gill-attachment          392685
gill-spacing             943507
gill-color                   49
stem-height                   0
stem-width                    0
stem-root               2067772
stem-surface            1486230
stem-color                   27
veil-type               2218034
veil-color              2055885
has-ring                     16
ring-type                 96564
spore-print-color       2137374
habitat                      36
season                        0
dtype: int64

In [16]:
test_df.isnull().sum()

cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [17]:
# Check for missing values

missing_values = raw_train_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage = (missing_values / len(raw_train_df)) * 100
missing_percentage

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.529421
cap-color                0.000428
does-bruise-or-bleed     0.000257
gill-attachment         16.797864
gill-spacing            40.360344
gill-color               0.002096
stem-height              0.000000
stem-width               0.000000
stem-root               88.452963
stem-surface            63.576375
stem-color               0.001155
veil-type               94.880712
veil-color              87.944474
has-ring                 0.000684
ring-type                4.130713
spore-print-color       91.430324
habitat                  0.001540
season                   0.000000
dtype: float64

In [18]:
# Check for missing values

missing_values = validation_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage = (missing_values / len(validation_df)) * 100
missing_percentage

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.524645
cap-color                0.000257
does-bruise-or-bleed     0.000257
gill-attachment         16.843528
gill-spacing            40.414919
gill-color               0.001027
stem-height              0.000000
stem-width               0.000000
stem-root               88.452037
stem-surface            63.476324
stem-color               0.001412
veil-type               94.895263
veil-color              87.914460
has-ring                 0.001027
ring-type                4.147134
spore-print-color       91.410957
habitat                  0.001155
season                   0.000000
dtype: float64

In [19]:
# Check for missing values

missing_values_test = test_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage_test = (missing_values_test / len(test_df)) * 100
missing_percentage_test

cap-diameter             0.000337
cap-shape                0.001492
cap-surface             21.506821
cap-color                0.000626
does-bruise-or-bleed     0.000481
gill-attachment         16.834796
gill-spacing            40.404694
gill-color               0.002358
stem-height              0.000048
stem-width               0.000000
stem-root               88.452543
stem-surface            63.595327
stem-color               0.001011
veil-type               94.878689
veil-color              87.880445
has-ring                 0.000914
ring-type                4.148051
spore-print-color       91.417224
habitat                  0.001203
season                   0.000000
dtype: float64

### Data Preprocessing
- Droping columns with more than 50% missing values
- Using simple imputer
- Encoding Categorical Variables

In [20]:
# Drop columns with extremely high missing values
columns_to_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color', 'stem-surface']
raw_train_df.drop(columns=columns_to_drop, inplace=True)
validation_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# # Check for missing values

# missing_values = train_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage = (missing_values / len(train_df)) * 100
# missing_percentage

In [21]:
# # Check for missing values

# missing_values_test = test_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df)) * 100
# missing_percentage_test

In [22]:
# Spliting dependent and independent valriable

y_train = raw_train_df['class']
raw_train_df = raw_train_df.drop('class', axis = 1)

raw_train_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
748879,2.45,x,g,n,f,a,NaN,p,7.12,2.30,w,f,f,d,u
589395,2.71,b,NaN,k,f,a,NaN,k,7.78,2.33,g,f,f,g,u


In [23]:
# Spliting dependent and independent valriable

y_val = validation_df['class']
validation_df = validation_df.drop('class', axis = 1)

validation_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
1706883,1.40,x,g,y,f,d,d,n,1.56,2.43,n,f,f,d,a
1554994,5.35,f,h,g,f,s,d,w,4.75,9.63,g,f,f,d,u


In [24]:
# Identify numerical and categorical columns
numerical_cols = raw_train_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = raw_train_df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print("Unique Value is each categorical column :")
    print(col, raw_train_df[col].nunique())

Unique Value is each categorical column :
cap-shape 61
Unique Value is each categorical column :
cap-surface 70
Unique Value is each categorical column :
cap-color 60
Unique Value is each categorical column :
does-bruise-or-bleed 24
Unique Value is each categorical column :
gill-attachment 69
Unique Value is each categorical column :
gill-spacing 44
Unique Value is each categorical column :
gill-color 54
Unique Value is each categorical column :
stem-color 47
Unique Value is each categorical column :
has-ring 21
Unique Value is each categorical column :
ring-type 33
Unique Value is each categorical column :
habitat 48
Unique Value is each categorical column :
season 4


In [25]:
# Define the imputer for numerical columns (KNN imputation)
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 5)),
    ('scaler', StandardScaler())
])

# Function to encode categorical columns with LabelEncoder
def encode_labels(df, columns):
    df_encoded = df.copy()
    le_dict = {}
    for col in columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        le_dict[col] = le
    return df_encoded, le_dict

# Function to decode categorical columns with LabelEncoder
def decode_labels(df, le_dict):
    df_decoded = df.copy()
    for col, le in le_dict.items():
        df_decoded[col] = le.inverse_transform(df_decoded[col])
    return df_decoded

# Define the imputer and encoder for categorical columns using KNNImputer
def impute_and_encode_labels_pipeline(X):
    df = pd.DataFrame(X, columns=categorical_cols)
    # Encode to numerical values
    df_encoded, le_dict = encode_labels(df, categorical_cols)
    # Impute missing values
    imputer = KNNImputer(n_neighbors = 5)
    df_imputed = pd.DataFrame(imputer.fit_transform(df_encoded), columns=categorical_cols)
    # Decode back to original categories
    df_decoded = decode_labels(df_imputed.round().astype(int), le_dict)
    # Encode again for the model
    df_reencoded, _ = encode_labels(df_decoded, categorical_cols)
    return df_reencoded.values

# Create a ColumnTransformer for numerical and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', FunctionTransformer(func=lambda X: impute_and_encode_labels_pipeline(X), validate=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Fit the preprocessor on the training data
preprocessor.fit(raw_train_df)

# Transform both the training and test data
train_df_transformed = preprocessor.transform(raw_train_df)
validation_df_transformed = preprocessor.transform(validation_df)
test_df_transformed = preprocessor.transform(test_df)

print("Transformed Training Data Shape:", train_df_transformed.shape)
print("Transformed Validation Data Shape:", validation_df_transformed.shape)
print("Transformed Test Data Shape:", test_df_transformed.shape)

logger.info(f"Missing values and categorical columns treatment completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Missing values and categorical columns treatment completed. Time elapsed: 68.51 seconds


Transformed Training Data Shape: (2337708, 15)
Transformed Validation Data Shape: (779237, 15)
Transformed Test Data Shape: (2077964, 15)


In [26]:
# Combine feature names
all_feature_names = numerical_cols.tolist() + categorical_cols.tolist()

# Convert to DataFrame
train_df_transformed = pd.DataFrame(train_df_transformed, columns=all_feature_names)
validation_df_transformed = pd.DataFrame(validation_df_transformed, columns=all_feature_names)
test_df_transformed = pd.DataFrame(test_df_transformed, columns=all_feature_names)

# # Check for missing values
# missing_values_train = train_df_transformed.isnull().sum()

# # Calculate the percentage of missing values
# missing_values_train = (missing_values_train / len(train_df_transformed)) * 100
# missing_values_train

In [27]:
# # Check for missing values

# missing_values_test = test_df_transformed.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df_transformed)) * 100
# missing_percentage_test

In [28]:
train_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-0.829222,0.285999,-1.093682,58.0,46.0,45.0,8.0,37.0,44.0,39.0,43.0,3.0,12.0,21.0,2.0
1,-0.773366,0.530507,-1.089977,36.0,70.0,42.0,8.0,37.0,44.0,34.0,27.0,3.0,12.0,25.0,2.0


In [29]:
validation_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-1.054793,-1.773796,-1.077626,35.0,16.0,41.0,3.0,10.0,7.0,23.0,24.0,5.0,12.0,7.0,0.0
1,-0.206215,-0.592007,-0.188388,21.0,17.0,25.0,3.0,24.0,7.0,32.0,17.0,5.0,12.0,7.0,2.0


In [30]:
test_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,0.500575,1.771571,0.736668,59.0,59.0,44.0,18.0,66.0,35.0,52.0,51.0,17.0,15.0,16.0,0.0
1,0.126771,-1.881231,-0.050062,50.0,53.0,45.0,5.0,66.0,17.0,54.0,38.0,6.0,14.0,16.0,0.0


In [31]:
# Feature Selection
def feature_selection(X_train, y_train, model):
    selector = SelectFromModel(model, threshold='mean')
    selector.fit(X_train, y_train)
    return selector.transform(X_train), selector.get_support()

# Initialize models
models = {
    # 'Logistic Regression': LogisticRegression(),
    # 'Ridge Classifier': RidgeClassifier(),
    # 'Decision Tree': DecisionTreeClassifier(),
    # 'Bagging Classifier': BaggingClassifier(),
    # 'Random Forest': RandomForestClassifier(),
    # 'AdaBoost Classifier': AdaBoostClassifier(),
    # 'Gradient Boosting': GradientBoostingClassifier(),
    # 'SVC': SVC(),
    # 'KNN': KNeighborsClassifier(),
    # 'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

# Define parameter grids for RandomizedSearchCV
param_grids = {
    'Logistic Regression': {
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
        'penalty': ['l2', 'none'],  # 'elasticnet' is not supported by 'lbfgs'
        'C': uniform(0.001, 1000),  # Inverse of regularization strength
        'max_iter': [100, 200, 300]
    },
    'Ridge Classifier': {
        'alpha': uniform(0.1, 10),  # Regularization strength
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg'],
        'fit_intercept': [True, False],
        'max_iter': [100, 200, 300],
       # 'normalize': [True, False]  # This should be removed as it's not a valid parameter
    },
    'Decision Tree': {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': randint(3, 20),
    'min_samples_split': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_samples_leaf': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_weight_fraction_leaf': uniform(0.0, 0.1),  # Random float between 0.0 and 0.1
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_leaf_nodes': randint(10, 50),
    'min_impurity_decrease': uniform(0.0, 0.1),
    'class_weight': [None, 'balanced']
},
    'Bagging Classifier': {
    'base_estimator': [DecisionTreeClassifier(), None],  # Default is DecisionTreeClassifier
    'n_estimators': randint(10, 100),
    'max_samples': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'max_features': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42]  # Set to None for random results, or a fixed integer for reproducibility
},
    'Random Forest': {
    'n_estimators': randint(50, 300),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, randint(3, 20)],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', uniform(0.5, 0.5)],
    'bootstrap': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42],
    'verbose': [0, 1],
    'warm_start': [True, False],
    'class_weight': [None, 'balanced']
},
    'AdaBoost Classifier': {
    'base_estimator': [None, DecisionTreeClassifier(max_depth=1)],
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 1.0),
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [42]
},
    'Gradient Boosting': {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.5),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None, uniform(0.1, 0.9)],
    'subsample': uniform(0.5, 0.5),
    'criterion': ['friedman_mse', 'squared_error'],
    #'alpha': uniform(0.5, 0.5),
    'random_state': [42],
    'verbose': [0, 1]
},
    'SVC': {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': randint(2, 5),
    'gamma': ['scale', 'auto', uniform(0.001, 1)],
    'coef0': uniform(0, 10),
    'shrinking': [True, False],
    'probability': [True, False],
    'tol': uniform(1e-5, 1e-1),
    'cache_size': uniform(50, 500),
    'class_weight': [None, 'balanced'],
    'verbose': [True, False],
    'max_iter': [-1, 100, 200],
    'decision_function_shape': ['ovr', 'ovo'],
    'break_ties': [True, False],
    'random_state': [42]
},
    'KNN': {
    'n_neighbors': randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': randint(1, 3),
    'metric': ['minkowski', 'euclidean', 'manhattan', 'chebyshev', 'hamming'],
    'leaf_size': randint(10, 50),
    'n_jobs': [None, -1]
},
    'XGBoost': {
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 0.5),
    'min_child_weight': randint(1, 10),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10),
    'max_delta_step': randint(0, 10),
    'colsample_bylevel': uniform(0.5, 0.5),
    'colsample_bynode': uniform(0.5, 0.5)
},
    'LightGBM': {
    'num_leaves': randint(20, 100),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 300),
    'min_child_samples': randint(10, 100),
    'min_split_gain': uniform(0, 0.5),
    'subsample': uniform(0.5, 0.5),
    'subsample_freq': randint(1, 10),
    'colsample_bytree': uniform(0.5, 0.5),
    'colsample_bylevel': uniform(0.5, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10)
}
}

# Optuna objective function for XGBoost and LightGBM
def objective(trial, model_name, model, param_grid, X_train, y_train):
    if model_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_uniform('subsample', 0.7, 0.9),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),
            'gamma': trial.suggest_uniform('gamma', 0, 0.2),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
        }
    elif model_name == 'LightGBM':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 31, 127),
            'max_depth': trial.suggest_int('max_depth', -1, 10),
            'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 0.2),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 0.2)
        }

    model.set_params(**params)
    model.fit(X_train, y_train)
    preds = model.predict(validation_df_transformed)
    score = matthews_corrcoef(y_val, preds)
    return score

# Hyperparameter tuning using RandomizedSearchCV
def tune_model(model_name, model, param_grid, X_train, y_train):
    search = RandomizedSearchCV(
        model, param_distributions=param_grid, n_iter=10, scoring='f1', cv=3, random_state=42
    )
    search.fit(X_train, y_train)
    return search.best_estimator_, search.best_params_

# Hyperparameter tuning using Optuna for XGBoost and LightGBM
def optuna_tune_model(model_name, model, X_train, y_train):
    study = optuna.create_study(direction='maximize', sampler=TPESampler())
    study.optimize(lambda trial: objective(trial, model_name, model, None, X_train, y_train), n_trials=50)
    return study.best_params_

# Train and evaluate models
def evaluate_models(X_train, y_train, X_val, y_val):
    best_models = {}
    for name, model in models.items():
        logger.info(f"Evaluating {name}")

        if name in ['XGBoost', 'LightGBM']:
            # Hyperparameter tuning using Optuna
            params = optuna_tune_model(name, model, X_train, y_train)
            model.set_params(**params)
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            score = matthews_corrcoef(y_val, preds)
            logger.info(f"{name} - MCC Score: {score:.4f}")
            logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")
            best_models[name] = model
        else:
            # Hyperparameter tuning using RandomizedSearchCV
            best_model, best_params = tune_model(name, model, param_grids[name], X_train, y_train)
            preds = best_model.predict(X_val)
            score = matthews_corrcoef(y_val, preds)
            logger.info(f"{name} - Best Params: {best_params}")
            logger.info(f"{name} - MCC Score: {score:.4f}")
            logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")
            best_models[name] = best_model

    return best_models

# Testing best models on the test set
def test_best_models(best_models, X_test, y_test):
    results = {}
    for name, model in best_models.items():
        preds = model.predict(X_test)
        score = matthews_corrcoef(y_test, preds)
        results[name] = score
        logger.info(f"Test MCC Score for {name}: {score:.4f}")
    return results

In [ ]:
# Encode the target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [32]:
# Evaluate models
best_models = evaluate_models(train_df_transformed, y_train, validation_df_transformed, y_val)

logger.info(f"Model evaluation completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Evaluating LightGBM
[I 2024-08-07 18:42:49,190] A new study created in memory with name: no-name-4dc80297-c915-44e6-9975-a865e936ad8d


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.152424 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 925
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[W 2024-08-07 18:43:12,816] Trial 0 failed with parameters: {'n_estimators': 102, 'learning_rate': 0.04856281719455987, 'num_leaves': 53, 'max_depth': -1, 'subsample': 0.9008194441814434, 'colsample_bytree': 0.92330992607079, 'reg_alpha': 0.15578043013539522, 'reg_lambda': 0.10190439037013783} because of the following error: NameError("name 'X_val_split' is not defined").
Traceback (most recent call last):
  File "C:\Users\younu\anaconda3\envs\gpu_env2\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\younu\AppData\Local\Temp\ipykernel_8396\2326508877.py", line 194, in <lambda>
    study.optimize(lambda trial: objective(trial, model_name, model, None, X_train, y_train), n_trials=50)
  File "C:\Users\younu\AppData\Local\Temp\ipykernel_8396\2326508877.py", line 179, in objective
    preds = model.predict(X_val_split)
NameError: name 'X_val_split' is not defined
[W 2024-08-07 18:43:12,816] Trial 0 failed with value Non

NameError: name 'X_val_split' is not defined

INFO:__main__:Evaluating Logistic Regression
INFO:__main__:Logistic Regression - Best Params: {'C': 374.54111884736244, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
INFO:__main__:Logistic Regression - MCC Score: 0.2327
INFO:__main__:Time elapsed: 5055.08 seconds
INFO:__main__:Evaluating Ridge Classifier
INFO:__main__:Ridge Classifier - Best Params: {'alpha': 3.845401188473625, 'fit_intercept': True, 'max_iter': 300, 'solver': 'cholesky'}
INFO:__main__:Ridge Classifier - MCC Score: 0.2332
INFO:__main__:Time elapsed: 5342.57 seconds
INFO:__main__:Evaluating Decision Tree
INFO:__main__:Decision Tree - Best Params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 17, 'max_features': 'log2', 'max_leaf_nodes': 17, 'min_impurity_decrease': 0.05986584841970366, 'min_samples_leaf': 0.025601864044243652, 'min_samples_split': 0.025599452033620268, 'min_weight_fraction_leaf': 0.005808361216819946, 'splitter': 'random'}
INFO:__main__:Decision Tree - MCC Score: 0.0000
INFO:__main__:Time elapsed: 5536.35 seconds
INFO:__main__:Evaluating Bagging Classifier
INFO:__main__:Bagging Classifier - Best Params: {'base_estimator': DecisionTreeClassifier(), 'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5917173949330818, 'max_samples': 0.8898455001363846, 'n_estimators': 30, 'n_jobs': None, 'oob_score': False, 'random_state': 42}
INFO:__main__:Bagging Classifier - MCC Score: 0.9804
INFO:__main__:Time elapsed: 13779.59 seconds
INFO:__main__:Evaluating Random Forest
INFO:__main__:Random Forest - Best Params: {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 171, 'n_jobs': None, 'oob_score': True, 'random_state': 42, 'verbose': 0, 'warm_start': True}
INFO:__main__:Random Forest - MCC Score: 0.9818
INFO:__main__:Time elapsed: 23825.43 seconds
INFO:__main__:Evaluating AdaBoost Classifier
INFO:__main__:AdaBoost Classifier - Best Params: {'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(max_depth=1), 'learning_rate': 0.9607143064099162, 'n_estimators': 156, 'random_state': 42}
INFO:__main__:AdaBoost Classifier - MCC Score: 0.5079
INFO:__main__:Time elapsed: 54563.87 seconds
INFO:__main__:Evaluating Gradient Boosting

In [ ]:
# Assuming you have y_test for the test set
y_test = pd.read_csv('test_labels.csv', index_col='id')  # Load test labels if available
X_test_transformed = preprocessor.transform(test_df)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=all_feature_names)
test_results = test_best_models(best_models, X_test_transformed, y_test)

logger.info(f"All evaluations completed.")